In [1]:
import os
import glob
import torchaudio
from speechbrain.pretrained import EncoderClassifier
import torch
from tqdm import tqdm
import torch.nn.functional as F
import numpy

c:\Users\ruben\OneDrive\Documentos\GitHub\speech2speech\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
c:\Users\ruben\OneDrive\Documentos\GitHub\speech2speech\venv\Lib\site-packages\speechbrain\utils\torch_audio_backend.py:22: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
torchvision is not available - cannot save figures
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
c:\Users\ruben\OneDrive\Documentos\GitHub\speech2speech\venv\Lib\site-packages\speechbrain\utils\torch_audio_backend.py:22: UserWarning: torchau

## Embedder
This script takes a folder of wav files and creates a embedding version of all of them.

In [2]:

spk_model = {
    "speechbrain/spkrec-xvect-voxceleb": 512, 
    "speechbrain/spkrec-ecapa-voxceleb": 192,
}

def f2embed(wav_file, classifier, size_embed):
    signal, fs = torchaudio.load(wav_file)
    assert fs == 16000, fs
    with torch.no_grad():
        embeddings = classifier.encode_batch(signal)
        embeddings = F.normalize(embeddings, dim=2)
        embeddings = embeddings.squeeze().cpu().numpy()
    assert embeddings.shape[0] == size_embed, embeddings.shape[0]
    return embeddings

def process_all(dir_in, dir_out):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", run_opts={"device": device})#, savedir=os.path.join('/tmp', "speechbrain/spkrec-xvect-voxceleb"))

    for faudio in os.listdir(dir_in):
        if faudio.split('.')[-1]=='wav':
            embed = f2embed(os.path.join(dir_in, faudio), classifier, 512)
            numpy.save(os.path.join(dir_out, faudio.replace('.wav', '.npy')), embed)

def process(args):
    wavlst = []
    for split in args.splits.split(","):
        wav_dir = os.path.join(args.arctic_root, split)
        wavlst_split = glob.glob(os.path.join(wav_dir, "wav", "*.wav"))
        print(f"{split} {len(wavlst_split)} utterances.")
        wavlst.extend(wavlst_split)

    spkemb_root = args.output_root
    if not os.path.exists(spkemb_root):
        pass

In [3]:
process_all("audio", "embed")